# 模型微調(Fine tuning)

In [ ]:
# 如果上傳檔案有誤，可使用下列指令刪除
# 替換 @API-KEY 為你的 OpenAI API KEY
# 由上一格執行結果取得 @FILE_ID
# !curl https://api.openai.com/v1/files/@FILE_ID -X DELETE -H "Authorization: Bearer @API-KEY"

## 建立 OpenAI 用戶端

In [1]:
from openai import OpenAI
client = OpenAI()

## 上傳 jsonl 檔案至 to OpenAI，至少要有10筆，不可有空白行

In [3]:
# method 1
file_obj = client.files.create(
  file=open("chat_data.jsonl", "rb"),
  purpose="fine-tune"
)
file_obj

FileObject(id='file-Q4d9Mv7iYv19J184KdMxd8', bytes=1682, created_at=1737204421, filename='chat_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
# method 2
# 替換 @API-KEY 為你的 OpenAI API KEY
!curl https://api.openai.com/v1/files -H "Authorization: Bearer @API-KEY" -F purpose="fine-tune" -F file="@chat_data.jsonl"

## 送出訓練任務(Job)

In [4]:
file_obj.id

'file-Q4d9Mv7iYv19J184KdMxd8'

In [5]:
job = client.fine_tuning.jobs.create(
    training_file=file_obj.id,
    model="gpt-4o-mini-2024-07-18"
)
job

FineTuningJob(id='ftjob-dXKOOqljiGphAhO3fyJ0DOKD', created_at=1737204522, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-8b5wOBmK82BeorSFOMxwOTtc', result_files=[], seed=668140145, status='validating_files', trained_tokens=None, training_file='file-Q4d9Mv7iYv19J184KdMxd8', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

## 查詢模型訓練狀態

In [10]:
job = client.fine_tuning.jobs.retrieve(job.id)
job

FineTuningJob(id='ftjob-dXKOOqljiGphAhO3fyJ0DOKD', created_at=1737204522, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:digibank::Ar2eGYCW', finished_at=1737204926, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=10), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-8b5wOBmK82BeorSFOMxwOTtc', result_files=['file-QiHGkkEQAFkJ3tUvRQKqnf'], seed=668140145, status='succeeded', trained_tokens=2690, training_file='file-Q4d9Mv7iYv19J184KdMxd8', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=10)), type='supervised'), user_provided_suffix=None)

In [ ]:
## 取消訓練任務(Job)
# client.fine_tuning.jobs.cancel(job.id)

## 模型推論

In [13]:
completion = client.chat.completions.create(
    model=job.fine_tuned_model,
    messages=[
        # {"role": "system", "content": ""},
        {"role": "user", "content": "tell me the capital of UK."}
    ]
)

print(completion.choices[0].message.content)

London, as if everyone doesn't know that already.


## 刪除模型

In [ ]:
client.models.delete(job.fine_tuned_model)

ModelDeleted(id='ft:gpt-4o-mini-2024-07-18:digibank::Ar2eGYCW', deleted=True, object='model')

## 刪除已上傳的資料檔

In [15]:
client.files.delete(file_obj.id)

FileDeleted(id='file-Q4d9Mv7iYv19J184KdMxd8', deleted=True, object='file')